In [2]:
# =========================================================
# 📘 04_recommendation_system.ipynb
# Author: Apoorva Sharma
# Project: AI-Powered Precision Nutrition
# Purpose: Generate personalized, safe, and optimized meal recommendations
# =========================================================

import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# ---------------------------------------------------------
# 1️⃣ Setup paths
# ---------------------------------------------------------
BASE_DIR = Path("D:/Complete_Data/nutrition_project")
PROCESSED_DIR = BASE_DIR / "processed_data"
RESULTS_DIR = BASE_DIR / "results"

SAFE_RECIPES_PATH = PROCESSED_DIR / "recipes_safe.csv"
OPTIMIZED_PLANS_PATH = RESULTS_DIR / "optimized_meal_plans.csv"
OUTPUT_RECOMMENDATIONS = RESULTS_DIR / "recommendation_samples.csv"

# ---------------------------------------------------------
# 2️⃣ Load datasets
# ---------------------------------------------------------
recipes = pd.read_csv(SAFE_RECIPES_PATH)
plans = pd.read_csv(OPTIMIZED_PLANS_PATH)

print(f"✅ Loaded safe recipes: {recipes.shape}")
print(f"✅ Loaded optimized meal plans: {plans.shape}")

# ---------------------------------------------------------
# 3️⃣ Feature preparation (nutrients + text)
# ---------------------------------------------------------
# Select nutrient columns
nutrient_cols = [
    "energy_kcal_mean_y",
    "protein_g_mean_y",
    "fat_g_mean_y",
    "carbs_g_mean_y",
    "Total_emissions_mean_y"
]
nutrient_cols = [c for c in nutrient_cols if c in recipes.columns]

nutrient_data = recipes[nutrient_cols].fillna(recipes[nutrient_cols].mean())

scaler = MinMaxScaler()
nutrient_scaled = scaler.fit_transform(nutrient_data)

# ---------------------------------------------------------
# 4️⃣ Ingredient text vectorization (robust version)
# ---------------------------------------------------------
# Ensure 'ingredient_text_clean' exists
if "ingredient_text_clean" not in recipes.columns:
    possible_cols = [c for c in recipes.columns if "ingredient" in c.lower()]
    if possible_cols:
        recipes["ingredient_text_clean"] = recipes[possible_cols[0]].astype(str)
    else:
        recipes["ingredient_text_clean"] = ""

# Clean text and handle empty values
recipes["ingredient_text_clean"] = (
    recipes["ingredient_text_clean"]
    .fillna("")
    .astype(str)
    .apply(lambda x: x if x.strip() != "" else "unknown_ingredient")
)

# Handle case where column is entirely blank
if recipes["ingredient_text_clean"].str.strip().eq("").all():
    print("⚠️ Warning: ingredient_text_clean is empty — using dummy tokens.")
    recipes["ingredient_text_clean"] = ["unknown_ingredient"] * len(recipes)

vectorizer = CountVectorizer(max_features=1000, stop_words="english")

try:
    ingredient_features = vectorizer.fit_transform(recipes["ingredient_text_clean"]).toarray()
    print(f"✅ Ingredient features extracted: {ingredient_features.shape}")
except ValueError:
    print("⚠️ Ingredient text empty after preprocessing — using zero vector fallback.")
    ingredient_features = np.zeros((len(recipes), 1000))

# ---------------------------------------------------------
# 5️⃣ Combine features (hybrid)
# ---------------------------------------------------------
combined_features = np.hstack([
    nutrient_scaled * 0.6,  # Nutrient importance weight
    ingredient_features * 0.4  # Ingredient-based similarity
])

# ---------------------------------------------------------
# 6️⃣ Compute recipe similarity
# ---------------------------------------------------------
similarity_matrix = cosine_similarity(combined_features)
print(f"✅ Similarity matrix shape: {similarity_matrix.shape}")

# ---------------------------------------------------------
# 7️⃣ Recommendation function
# ---------------------------------------------------------
def recommend_recipes(recipe_title, top_n=5):
    if recipe_title not in recipes["title"].values:
        print(f"⚠️ Recipe '{recipe_title}' not found.")
        return pd.DataFrame()
    
    idx = recipes[recipes["title"] == recipe_title].index[0]
    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_indices = [i[0] for i in sim_scores[1: top_n + 1]]

    recs = recipes.iloc[top_indices][["title"] + nutrient_cols].copy()
    recs["similarity_score"] = [sim_scores[i + 1][1] for i in range(top_n)]
    return recs

# ---------------------------------------------------------
# 8️⃣ Example recommendations
# ---------------------------------------------------------
sample_title = recipes["title"].iloc[0]
recommendations_df = recommend_recipes(sample_title, top_n=5)

if not recommendations_df.empty:
    print(f"🍽️ Recommendations for: {sample_title}")
    display(recommendations_df)
else:
    print("⚠️ No recommendations generated.")

# ---------------------------------------------------------
# 9️⃣ Save results
# ---------------------------------------------------------
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
recommendations_df.to_csv(OUTPUT_RECOMMENDATIONS, index=False)
print(f"💾 Recommendations saved to: {OUTPUT_RECOMMENDATIONS}")

print("✅ Recommendation pipeline completed successfully!")


✅ Loaded safe recipes: (17775, 25)
✅ Loaded optimized meal plans: (50, 5)
✅ Ingredient features extracted: (17775, 1)
✅ Similarity matrix shape: (17775, 17775)
🍽️ Recommendations for: "Adult" Pimiento Cheese 


title  energy_kcal_mean_y  \
1                              "Blanketed" Eggplant           220.243118   
2  "Bloody Mary" Tomato Toast with Celery and Hor...          220.243118   
3                        "Brown on Blonde" Blondies           220.243118   
4                           "California Roll" Salad           220.243118   
5             "Candy Corn" Frozen Citrus Cream Pops           220.243118   

   protein_g_mean_y  fat_g_mean_y  carbs_g_mean_y  Total_emissions_mean_y  \
1         11.926873     10.687361       19.569788                1.373454   
2         11.926873     10.687361       19.569788                1.373454   
3         11.926873     10.687361       19.569788                1.373454   
4         11.926873     10.687361       19.569788                1.373454   
5         11.926873     10.687361       19.569788                1.373454   

   similarity_score  
1               1.0  
2               1.0  
3               1.0  
4               1.0  
5               1.0

💾 Recommendations saved to: D:\Complete_Data\nutrition_project\results\recommendation_samples.csv
✅ Recommendation pipeline completed successfully!
